# Preprocessing

This notebook is used for the data preprocessing of the Ethereum transactions dataset. Per the Team Goal framework, a dataset engineered using Principal Component Analysis (PCA) is needed. This dataset is created in the 'Preprocessing - Simple PCA' section. Furthermore, in order for Ben to achieve his 3 Domain Specific Goals he will be engineering 3 other datasets. A description of the dataset created in each section is as follows:

'Preprocess - Base' = minimally preprocessed base dataset. Given their datastructure, some of the features in the original dataset must be augmented in some way prior to using PCA and other preprocessing/modeling techniques.

'Preprocess - Base Standardized' = Ben's Domain Specific Goal. Apply standardization to the Base dataset.

'Preprocess - PCA' = Team Goal & Ben's Domain Specific Goal. Apply standardization then PCA to the base dataset

In [16]:
# import libraries
import pandas as pd
import numpy as np

In [17]:
# get dataset 
data = pd.read_csv('Dataset.csv')
data.head()

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,block_timestamp,block_number,block_hash,from_scam,to_scam,from_category,to_category
0,0x253ec84729f5c11437f5346830e6bdc9857171b16097...,370394,4,0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0,0xd707ac0098599006f0857e8da4c950795645ba01,1.800000e+18,30000,2.205000e+10,0x,129358,21000,2017-10-16 05:26:53 UTC,4370008,0x5e14d30d2155c0cdd65044d7e0f296373f3e92f65ebd...,0,0,NaN,NaN
1,0xfa5f1d7715562adb1e408fdbbbbfc033ae4fadca1e07...,229082,85,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xb8f6e76d34d4877732dfefa6b6a4e2a834a9092b,7.500000e+15,90000,2.060947e+10,0x,5415387,21000,2017-10-16 05:28:49 UTC,4370014,0x900efe010b41b8b6b135db041422fac80b69b96d93c2...,0,0,NaN,NaN
2,0x37928f1d12ee769d0876695aaa815c13d705f8b1e376...,229095,31,0x304cc179719bc5b05418d6f7f6783abe45d83090,0x5ec492652d7b92421680f21169ff6a8dbbc622ab,7.500000e+15,90000,2.060947e+10,0x,1369625,21000,2017-10-16 05:29:19 UTC,4370015,0xfe7d1bc1c6257f92bacb0f8b1266c9894dda99b42e78...,0,0,NaN,NaN
3,0xaad886f331a2fb40495cdf5faa10882852fcd324f064...,229126,78,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xdf01b231b53d8b4cbb1b092a8dc86730cebf4aa9,7.500000e+15,90000,2.060947e+10,0x,2464655,21000,2017-10-16 05:29:43 UTC,4370016,0x1d132e9cd35e9e2cf8350258401b43474772eb1904f1...,0,0,NaN,NaN
4,0x3dbe36fd1a1b01b5e1b343ff78ec1ec189feee5acf46...,229131,83,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xaa4b9693c54029d180134161cc99e98b76cea249,7.500000e+15,90000,2.060947e+10,0x,2569655,21000,2017-10-16 05:29:43 UTC,4370016,0x1d132e9cd35e9e2cf8350258401b43474772eb1904f1...,0,0,NaN,NaN


# Preprocess - Base

In [18]:
# remove specific categories of fraud. May be useful for analysis of results but cannot be used for modeling
from_category = data.pop('from_category')
to_category = data.pop('to_category')

# remove fraud indication categories. Modeling targets
from_scam = data.pop('from_scam')
to_scam = data.pop('to_scam')

# save 
from_scam.to_pickle('Data/from_scam.pkl')
to_scam.to_pickle('Data/to_scam.pkl')
from_category.to_pickle('Data/from_category.pkl')
to_category.to_pickle('Data/to_category.pkl')

In [19]:
# decompose timestamp into a form readable for PCA / ML

times = data['block_timestamp']

# lists to be used as new features in data
years = []
months =[]
days =[]
hours = []
minutes = []
seconds =[]

for i in times:
    
    # partition string into array to split year, month and day/time
    splits = i.split(sep='-')
    year = splits[0] # get year
    month = splits[1]# get month
    
    # further partition day/time into day and time
    splits = splits[2].split(sep=' ')
    day = splits[0]  # get day
    
    # further partition time into minutes, hours, and seconds
    splits = splits[1].split(sep=':')
    hour = splits[0] # get hour
    minute = splits[1] # get minute
    second = splits[2][0:2] #get second
    
    # append each observation to corresponding feature list
    years.append(year)
    months.append(month)
    days.append(day)
    hours.append(hour)
    minutes.append(minute)
    seconds.append(second)

In [20]:
# insert new columns based on new datetime features
data['years'] = list(map(int,years))
data['months'] = list(map(int,months))
data['days'] = list(map(int,days))
data['hours'] = list(map(int,hours))
data['minutes'] = list(map(int,minutes)) 
data['seconds'] = list(map(int,seconds)) 
block_timestamp = data.pop('block_timestamp')

data.head()

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,block_number,block_hash,years,months,days,hours,minutes,seconds
0,0x253ec84729f5c11437f5346830e6bdc9857171b16097...,370394,4,0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0,0xd707ac0098599006f0857e8da4c950795645ba01,1.800000e+18,30000,2.205000e+10,0x,129358,21000,4370008,0x5e14d30d2155c0cdd65044d7e0f296373f3e92f65ebd...,2017,10,16,5,26,53
1,0xfa5f1d7715562adb1e408fdbbbbfc033ae4fadca1e07...,229082,85,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xb8f6e76d34d4877732dfefa6b6a4e2a834a9092b,7.500000e+15,90000,2.060947e+10,0x,5415387,21000,4370014,0x900efe010b41b8b6b135db041422fac80b69b96d93c2...,2017,10,16,5,28,49
2,0x37928f1d12ee769d0876695aaa815c13d705f8b1e376...,229095,31,0x304cc179719bc5b05418d6f7f6783abe45d83090,0x5ec492652d7b92421680f21169ff6a8dbbc622ab,7.500000e+15,90000,2.060947e+10,0x,1369625,21000,4370015,0xfe7d1bc1c6257f92bacb0f8b1266c9894dda99b42e78...,2017,10,16,5,29,19
3,0xaad886f331a2fb40495cdf5faa10882852fcd324f064...,229126,78,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xdf01b231b53d8b4cbb1b092a8dc86730cebf4aa9,7.500000e+15,90000,2.060947e+10,0x,2464655,21000,4370016,0x1d132e9cd35e9e2cf8350258401b43474772eb1904f1...,2017,10,16,5,29,43
4,0x3dbe36fd1a1b01b5e1b343ff78ec1ec189feee5acf46...,229131,83,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xaa4b9693c54029d180134161cc99e98b76cea249,7.500000e+15,90000,2.060947e+10,0x,2569655,21000,4370016,0x1d132e9cd35e9e2cf8350258401b43474772eb1904f1...,2017,10,16,5,29,43


In [21]:
# NOTE: input messages are hashed and therefore cannot be decoded. Thus any NLP on the messages is 
# not possible and including a representation of the hashes would be adding random noise
# thereby making an indicator feature the most appropriate representation

# map input feature to binary indicator value. 0 if no additional message was sent with transaction,
# 1 if additional message was sent with transaction

input_keys = {'0x':0} # map transactions with no additional message to 0
test = data.input.map(input_keys)
test.unique()

# map NA values (trasnactions with additional message) to 1
test = test.fillna(1)

data.input = list(map(int,test))
data.head()

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,block_number,block_hash,years,months,days,hours,minutes,seconds
0,0x253ec84729f5c11437f5346830e6bdc9857171b16097...,370394,4,0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0,0xd707ac0098599006f0857e8da4c950795645ba01,1.800000e+18,30000,2.205000e+10,0,129358,21000,4370008,0x5e14d30d2155c0cdd65044d7e0f296373f3e92f65ebd...,2017,10,16,5,26,53
1,0xfa5f1d7715562adb1e408fdbbbbfc033ae4fadca1e07...,229082,85,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xb8f6e76d34d4877732dfefa6b6a4e2a834a9092b,7.500000e+15,90000,2.060947e+10,0,5415387,21000,4370014,0x900efe010b41b8b6b135db041422fac80b69b96d93c2...,2017,10,16,5,28,49
2,0x37928f1d12ee769d0876695aaa815c13d705f8b1e376...,229095,31,0x304cc179719bc5b05418d6f7f6783abe45d83090,0x5ec492652d7b92421680f21169ff6a8dbbc622ab,7.500000e+15,90000,2.060947e+10,0,1369625,21000,4370015,0xfe7d1bc1c6257f92bacb0f8b1266c9894dda99b42e78...,2017,10,16,5,29,19
3,0xaad886f331a2fb40495cdf5faa10882852fcd324f064...,229126,78,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xdf01b231b53d8b4cbb1b092a8dc86730cebf4aa9,7.500000e+15,90000,2.060947e+10,0,2464655,21000,4370016,0x1d132e9cd35e9e2cf8350258401b43474772eb1904f1...,2017,10,16,5,29,43
4,0x3dbe36fd1a1b01b5e1b343ff78ec1ec189feee5acf46...,229131,83,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xaa4b9693c54029d180134161cc99e98b76cea249,7.500000e+15,90000,2.060947e+10,0,2569655,21000,4370016,0x1d132e9cd35e9e2cf8350258401b43474772eb1904f1...,2017,10,16,5,29,43


In [22]:
# Remove (and store) columns that will not be used 

# Here again, the transaction and block hashes are random noise so they should not be included
transaction_hash = data.pop('hash') # transaction hash is random, thus cannot be useful for prediction
block_hash = data.pop('block_hash') # block hash is random, thus cannot be useful for predictions

# In our case, addresses should not be used for our prediction task because we are trying to identify 
# fraudulent transactions from entities that are not already known to be malicious. On a transaction to
# transaction basis the identities of the ostensibly honest entities involved do not matter it is simply
# the transaction that matters. It is only once fraud is identified that these identities matter for 
# penalty enforcement. Furthermore, the address is a random hash and adds no useful information unless 
# we know a priori that the address is malicious.

from_address = data.pop('from_address') 
to_address = data.pop('to_address')
data.head()

,nonce,transaction_index,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,block_number,years,months,days,hours,minutes,seconds
0,370394,4,1.800000e+18,30000,2.205000e+10,0,129358,21000,4370008,2017,10,16,5,26,53
1,229082,85,7.500000e+15,90000,2.060947e+10,0,5415387,21000,4370014,2017,10,16,5,28,49
2,229095,31,7.500000e+15,90000,2.060947e+10,0,1369625,21000,4370015,2017,10,16,5,29,19
3,229126,78,7.500000e+15,90000,2.060947e+10,0,2464655,21000,4370016,2017,10,16,5,29,43
4,229131,83,7.500000e+15,90000,2.060947e+10,0,2569655,21000,4370016,2017,10,16,5,29,43


# Preprocess - Base Standardized

In [23]:
# Standardize base datset features

import sklearn
from sklearn.preprocessing import StandardScaler

# value
transformer = StandardScaler().fit(np.array(data['value']).reshape(-1,1))
transformed_value = transformer.transform(np.array(data['value']).reshape(-1,1)).reshape(1,71250)[0]
data['value']=transformed_value

# nonce
transformer = StandardScaler().fit(np.array(data['nonce']).reshape(-1,1))
transformed_nonce = transformer.transform(np.array(data['nonce']).reshape(-1,1)).reshape(1,71250)[0]
data['nonce'] = transformed_nonce

# transaction_index
transformer = StandardScaler().fit(np.array(data['transaction_index']).reshape(-1,1))
transformed_transaction_index = transformer.transform(np.array(data['transaction_index']).reshape(-1,1)).reshape(1,71250)[0]
data['transaction_index'] = transformed_transaction_index

# gas
transformer = StandardScaler().fit(np.array(data['gas']).reshape(-1,1))
transformed_gas = transformer.transform(np.array(data['gas']).reshape(-1,1)).reshape(1,71250)[0]
data['gas'] = transformed_gas

# gas_price
transformer = StandardScaler().fit(np.array(data['gas_price']).reshape(-1,1))
transformed_gas_price = transformer.transform(np.array(data['gas_price']).reshape(-1,1)).reshape(1,71250)[0]
data['gas_price'] = transformed_gas_price

# receipt_cumlative_gas_used
transformer = StandardScaler().fit(np.array(data['receipt_cumulative_gas_used']).reshape(-1,1))
transformed_cumlative_receipt_gas_used = transformer.transform(np.array(data['receipt_cumulative_gas_used']).reshape(-1,1)).reshape(1,71250)[0]
data['receipt_cumulative_gas_used'] = transformed_cumlative_receipt_gas_used


# receipt_gas_used
transformer = StandardScaler().fit(np.array(data['receipt_gas_used']).reshape(-1,1))
transformed_receipt_gas_used = transformer.transform(np.array(data['receipt_gas_used']).reshape(-1,1)).reshape(1,71250)[0]
data['receipt_gas_used'] = transformed_receipt_gas_used

# years
transformer = StandardScaler().fit(np.array(data['years']).reshape(-1,1))
transformed_years = transformer.transform(np.array(data['years']).reshape(-1,1)).reshape(1,71250)[0]
data['years'] = transformed_years

# months
transformer = StandardScaler().fit(np.array(data['months']).reshape(-1,1))
transformed_months = transformer.transform(np.array(data['months']).reshape(-1,1)).reshape(1,71250)[0]
data['months'] = transformed_months

# days
transformer = StandardScaler().fit(np.array(data['days']).reshape(-1,1))
transformed_days = transformer.transform(np.array(data['days']).reshape(-1,1)).reshape(1,71250)[0]
data['days'] = transformed_days

# hours
transformer = StandardScaler().fit(np.array(data['hours']).reshape(-1,1))
transformed_hours = transformer.transform(np.array(data['hours']).reshape(-1,1)).reshape(1,71250)[0]
data['hours'] = transformed_hours

# minutes
transformer = StandardScaler().fit(np.array(data['minutes']).reshape(-1,1))
transformed_minutes = transformer.transform(np.array(data['minutes']).reshape(-1,1)).reshape(1,71250)[0]
data['minutes'] = transformed_minutes

# seconds
transformer = StandardScaler().fit(np.array(data['seconds']).reshape(-1,1))
transformed_seconds = transformer.transform(np.array(data['seconds']).reshape(-1,1)).reshape(1,71250)[0]
data['seconds'] = transformed_minutes

# block number
transformer = StandardScaler().fit(np.array(data['block_number']).reshape(-1,1))
transformed_block = transformer.transform(np.array(data['block_number']).reshape(-1,1)).reshape(1,71250)[0]
data['block_number'] = transformed_block

data.head()

,nonce,transaction_index,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,block_number,years,months,days,hours,minutes,seconds
0,0.062658,-1.062556,-0.028777,-0.287106,-0.099530,0,-1.152749,-0.10271,-1.117982,-2.007005,1.54363,-0.230799,-0.934209,-0.164294,-0.164294
1,-0.078960,0.190348,-0.035048,0.162133,-0.111052,0,1.009971,-0.10271,-1.117974,-2.007005,1.54363,-0.230799,-0.934209,-0.049980,-0.049980
2,-0.078947,-0.644921,-0.035048,0.162133,-0.111052,0,-0.645307,-0.10271,-1.117972,-2.007005,1.54363,-0.230799,-0.934209,0.007177,0.007177
3,-0.078916,0.082072,-0.035048,0.162133,-0.111052,0,-0.197288,-0.10271,-1.117971,-2.007005,1.54363,-0.230799,-0.934209,0.007177,0.007177
4,-0.078911,0.159412,-0.035048,0.162133,-0.111052,0,-0.154328,-0.10271,-1.117971,-2.007005,1.54363,-0.230799,-0.934209,0.007177,0.007177


In [24]:
# view standardized data
data.head()

,nonce,transaction_index,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,block_number,years,months,days,hours,minutes,seconds
0,0.062658,-1.062556,-0.028777,-0.287106,-0.099530,0,-1.152749,-0.10271,-1.117982,-2.007005,1.54363,-0.230799,-0.934209,-0.164294,-0.164294
1,-0.078960,0.190348,-0.035048,0.162133,-0.111052,0,1.009971,-0.10271,-1.117974,-2.007005,1.54363,-0.230799,-0.934209,-0.049980,-0.049980
2,-0.078947,-0.644921,-0.035048,0.162133,-0.111052,0,-0.645307,-0.10271,-1.117972,-2.007005,1.54363,-0.230799,-0.934209,0.007177,0.007177
3,-0.078916,0.082072,-0.035048,0.162133,-0.111052,0,-0.197288,-0.10271,-1.117971,-2.007005,1.54363,-0.230799,-0.934209,0.007177,0.007177
4,-0.078911,0.159412,-0.035048,0.162133,-0.111052,0,-0.154328,-0.10271,-1.117971,-2.007005,1.54363,-0.230799,-0.934209,0.007177,0.007177


In [25]:
# save
data.to_pickle('Data/standardized_base.pkl')

# Preprocess - PCA

In [26]:
# remove value as it will not be included in PCA
val = data.pop('value')

In [27]:
# fit pca to standardized data and get variance explained by each component

from sklearn.decomposition import PCA
pca = PCA()
pca.fit(data)

n=0
for i in pca.explained_variance_ratio_:
    n+=1
    print("Component ",n," variance explained: ",i*100)

Component  1  variance explained:  15.595190451970971
Component  2  variance explained:  15.100294630405351
Component  3  variance explained:  13.598533834327409
Component  4  variance explained:  12.049820555753776
Component  5  variance explained:  9.140467005931146
Component  6  variance explained:  7.620143408519751
Component  7  variance explained:  7.390059999691033
Component  8  variance explained:  6.835841525539271
Component  9  variance explained:  6.607266645003968
Component  10  variance explained:  4.176357732222911
Component  11  variance explained:  1.2916014196322614
Component  12  variance explained:  0.5940244493580499
Component  13  variance explained:  0.00039834164410217245
Component  14  variance explained:  1.1906682092570576e-31


In [28]:
principal_components = pd.DataFrame(pca.transform(data))
principal_components['value'] = val
principal_components

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,value
0,-2.006815,-2.169782,0.146400,0.467598,-0.327913,0.751913,-0.521793,-0.761362,0.788030,0.315258,-0.054779,0.023103,-0.001547,1.587008e-17,-0.028777
1,-1.352242,-1.128665,-1.880299,0.636689,-0.067591,0.997769,-0.122053,-0.728717,0.883459,-0.048290,0.590403,0.036177,-0.002929,-1.502062e-16,-0.035048
2,-1.678475,-2.004593,-0.426661,0.696365,-0.140810,0.824183,-0.329805,-0.819126,0.714460,-0.011513,0.009796,0.033345,-0.001582,-2.395613e-17,-0.035048
3,-1.505144,-1.625462,-1.122661,0.626570,-0.059864,0.908407,-0.226611,-0.791670,0.777155,-0.030135,-0.182153,0.033123,-0.000876,5.063698e-17,-0.035048
4,-1.487477,-1.586732,-1.193951,0.619052,-0.051163,0.917079,-0.216019,-0.788997,0.783427,-0.032049,-0.205875,0.033085,-0.000792,5.931060e-17,-0.035048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71245,2.927621,3.952934,-1.212045,1.963278,-3.406708,0.519575,-1.005464,2.403966,0.841224,-0.720638,-0.439987,1.163798,-0.031265,5.102172e-17,-0.032790
71246,3.129705,2.166407,0.428769,2.182397,-3.596906,0.810269,-1.649254,2.565655,0.294736,-0.647513,-0.464002,1.152256,-0.029994,-8.957099e-17,-0.031261
71247,4.011104,2.980827,-1.680243,2.066867,-3.457940,0.991704,-1.343746,2.668197,0.494200,-0.703986,-0.225467,1.155607,-0.030298,-1.240845e-16,-0.031261
71248,3.864285,0.701578,1.355216,2.283595,-3.823042,-0.301002,-1.176659,2.635953,0.294222,-0.667384,-0.441713,1.182729,-0.030108,-2.256291e-16,-0.034906


In [29]:
principal_components.to_pickle('Data/principal_components.pkl')